# Remplissage d'une base de données existante

## Depuis Pandas

In [92]:
import sqlite3
import pandas as pd
#import pyodbc

In [93]:
base_GTFS = sqlite3.connect('./GTFS/gtfs_tag.db')

In [94]:
don_agency = pd.read_csv(
    './GTFS/SEM-GTFS/agency.txt', sep=",")

In [95]:
don_agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone
0,SEM,Mobilités M - Tag,https://www.mobilites-m.fr/,Europe/Paris,FR,438703870


In [96]:
don_stops = pd.read_csv(
    './GTFS/SEM-GTFS/stops.txt', sep=",")

In [97]:
don_stops

,stop_id,stop_code,stop_name,stop_lat,stop_lon,zone_id,location_type,parent_station,wheelchair_boarding
0,0004,0004,"Meylan, Revirée - Sayettes",45.21220,5.76728,NaN,NaN,REVIREES,0
1,0006,0006,"La Tronche, Commandant Nal",45.20323,5.74535,NaN,NaN,COMMANAL,1
2,0007,0007,"La Tronche, Musée Hébert",45.20564,5.74888,NaN,NaN,MUSHEBER,1
3,0008,0008,"La Tronche, Centre Théologique",45.20617,5.75214,NaN,NaN,CENTHEOL,1
4,0011,0011,"Meylan, Malacher",45.21128,5.78724,NaN,NaN,MALACHER,1
...,...,...,...,...,...,...,...,...,...
2379,VOHRIGEN,GENVOHRIGEN,"Vizille, Parc Vöhringen",45.07888,5.76947,NaN,1.0,NaN,0
2380,XAVJOUVI,GENXAVJOUVI,"Grenoble, Xavier Jouvin",45.19560,5.73009,NaN,1.0,NaN,0
2381,ZASPEYRE,GENZASPEYRE,"Vif, Z.A. des Speyres",45.05690,5.68184,NaN,1.0,NaN,0
2382,ZELLAMEL,GENZELLAMEL,"Saint-Martin-d'Hères, Zella Mehlis",45.16707,5.75354,NaN,1.0,NaN,0


In [98]:
don_stops.columns[0]

'stop_id'

In [99]:
don_stops.loc[0][0]

'0004'

In [100]:
don_agency.to_sql(name="Agency", con=base_GTFS, if_exists='append', index=False)

In [101]:
don_stops.to_sql(name="Stops", con=base_GTFS, if_exists='append', index=False)

## Fonction pour générer une commande insertion SQL

### une chaine de caractère

In [102]:
curseur = base_GTFS.cursor()

In [140]:
table_name = "Stops"

In [104]:
a_dict = {'stop_id': 9999,'stop_code': 9999, 'stop_name': '"Ma Maison"', 'stop_lat': 45.2122, 'stop_lon': 5.76728, 'zone_id': None, 'location_type': None, 'parent_station': 'LYON','wheelchair_boarding': 1}

In [105]:
def gen_insert_query(table_name:str, a_dict:dict):
    """Création d'une chaine de caractère à partir d'un dictionnaire"""
    
    requete_init = f"INSERT INTO {table_name}("
    requete_in = []
    for i in a_dict:
        requete_in.append(str(i))

    requete_in_str =""
    for i in requete_in:
        requete_in_str += f"{i},"

    requete_in_str = requete_in_str[:-1]
    requete_in2= ")VALUES(" 

    requete_in_str2 =""
    for i in requete_in:
        requete_in_str2 += f":{i},"
    requete_in_str2 = requete_in_str2[:-1]
    
    requete=requete_init+requete_in_str+requete_in2+requete_in_str2+")"
    
    return requete

Alternative Philo: 

mySql_insert_query = 'INSERT INTO '+table_name+' ('+', '.join(tuple(getList(a_dict)))+')'+' VALUES '+' (:'+', :'.join(tuple(getList(a_dict)))+')'
    return(mySql_insert_query)
    
def getList(dict):
    return dict.keys()

Alternative Roman

def gen_insert_query(table_name, a_dict):
    cols = tuple(a_dict.keys())
    vals = tuple(a_dict.values())
    return f"INSERT INTO {table_name} {cols} VALUES {vals}"

In [106]:
curseur.execute(gen_insert_query(table_name,a_dict),a_dict)

In [107]:
base_GTFS.commit()

### une liste de chaines de caractères

In [108]:
def get_insert_queries(tablename:str, df: pd.DataFrame):
    liste = []
    df_in = df.copy()
    
    a_dict = df_in.to_dict(orient="results")
    for dic in a_dict:
        liste.append(gen_insert_query(table_name, dic))
        
    return liste

In [130]:
# ALTERNATIVE ROMAN
def get_insert_queries2(table_name, df):
    querylist = []
    for row in range (len(df)):
        querylist.append(gen_insert_query(table_name, dict(df.iloc[row,:])))
    return querylist

In [156]:
# ALTERNATIVE KEVIN
def get_insert_queries3(table_name:str, df:pd.DataFrame) -> str:
        dico = df.to_dict(orient='list')
        return f"INSERT INTO {table_name} {tuple(dico.keys())} VALUES {','.join(str(i) for i in tuple(zip(*dico.values())))}"

In [109]:
don_stops2 = don_stops.head(5)

REQUETES IDENTIQUES SANS LES VALEURS (fonctionne pour le .execute)

In [110]:
liste_sql = get_insert_queries(table_name,don_stops2)
liste_sql

/tmp/ipykernel_3694/1272310425.py:5: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  a_dict = df_in.to_dict(orient="results")


['INSERT INTO Stops(stop_id,stop_code,stop_name,stop_lat,stop_lon,zone_id,location_type,parent_station,wheelchair_boarding)VALUES(:stop_id,:stop_code,:stop_name,:stop_lat,:stop_lon,:zone_id,:location_type,:parent_station,:wheelchair_boarding)',
 'INSERT INTO Stops(stop_id,stop_code,stop_name,stop_lat,stop_lon,zone_id,location_type,parent_station,wheelchair_boarding)VALUES(:stop_id,:stop_code,:stop_name,:stop_lat,:stop_lon,:zone_id,:location_type,:parent_station,:wheelchair_boarding)',
 'INSERT INTO Stops(stop_id,stop_code,stop_name,stop_lat,stop_lon,zone_id,location_type,parent_station,wheelchair_boarding)VALUES(:stop_id,:stop_code,:stop_name,:stop_lat,:stop_lon,:zone_id,:location_type,:parent_station,:wheelchair_boarding)',
 'INSERT INTO Stops(stop_id,stop_code,stop_name,stop_lat,stop_lon,zone_id,location_type,parent_station,wheelchair_boarding)VALUES(:stop_id,:stop_code,:stop_name,:stop_lat,:stop_lon,:zone_id,:location_type,:parent_station,:wheelchair_boarding)',
 'INSERT INTO Stops(

REQUETES COMPLETES !

In [157]:
liste_test = get_insert_queries3(table_name,don_stops2)
liste_test

"INSERT INTO Stops ('stop_id', 'stop_code', 'stop_name', 'stop_lat', 'stop_lon', 'zone_id', 'location_type', 'parent_station', 'wheelchair_boarding') VALUES ('0004', '0004', 'Meylan, Revirée - Sayettes', 45.2122, 5.76728, nan, nan, 'REVIREES', 0),('0006', '0006', 'La Tronche, Commandant Nal', 45.20323, 5.74535, nan, nan, 'COMMANAL', 1),('0007', '0007', 'La Tronche, Musée Hébert', 45.20564, 5.74888, nan, nan, 'MUSHEBER', 1),('0008', '0008', 'La Tronche, Centre Théologique', 45.20617, 5.75214, nan, nan, 'CENTHEOL', 1),('0011', '0011', 'Meylan, Malacher', 45.21128, 5.78724, nan, nan, 'MALACHER', 1)"

In [160]:
a_dict2 = don_stops2.to_dict(orient="list")

In [161]:
a_dict = don_stops2.to_dict(orient="list")

In [113]:
a_dict[0]

{'stop_id': '0004',
 'stop_code': '0004',
 'stop_name': 'Meylan, Revirée - Sayettes',
 'stop_lat': 45.2122,
 'stop_lon': 5.76728,
 'zone_id': nan,
 'location_type': nan,
 'parent_station': 'REVIREES',
 'wheelchair_boarding': 0}

### Enregistrement sur la Base de D

In [115]:
i=0
for sql_request in liste_sql:
    curseur.execute(sql_request, a_dict[i])
    base_GTFS.commit()
    i+=1

## Fonction pour créer un fichier SQL

In [178]:
def gen_insert_file(filename, tablename, df):
    '''
    procédure qui crée un fichier db
    filename : est le nom de fichier (e.g. insert_stops.db)
    tablename : le nom de la relation où nous allons insérer nos tuples (e.g. gtfs_stops)
    df : un dataframe pandas
    '''
    fichier = open(f"{filename}", "a")
    liste = get_insert_queries3(tablename,df)
    fichier.write(f"\n{liste}")
    fichier.close()

In [164]:
don_stop_times = pd.read_csv(
    './GTFS/SEM-GTFS/stop_times.txt', sep=",")

In [165]:
don_trips = pd.read_csv(
    './GTFS/SEM-GTFS/trips.txt', sep=",")

### Création de deux fichiers sql

In [180]:
gen_insert_file("stop_times.sql", "StopTimes",don_stop_times)
gen_insert_file("trip.sql", "Trips",don_trips)